In [1]:
#Import the required libraries

import pandas as pd
import numpy as np
# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns
# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math
import matplotlib.pyplot as plt
# For missing values
import missingno as msg
#For datetime
import datetime
# For handling warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# read the engineered dataset

df=pd.read_csv("engineered_train.csv",encoding='utf-8')
df.head()

,body,downs,created_utc,score,author,distinguished,archived,subreddit,author_flair_css_class,author_flair_text,...,ups,controversiality,edited,body_clean,positive_sentiment,neutral_sentiment,negative_sentiment,compound_sentiment,nb_chars,nb_words
0,Most of us have some family members like this....,0,1420070400,14,YoungModern,NaN,False,exmormon,NaN,NaN,...,14,0.0,True,family member like family like,0.625,0.375,0.000,0.6124,30,5
1,But Mill's career was way better. Bentham is l...,0,1420070400,3,RedCoatsForever,NaN,False,CanadaPolitics,on,Ontario,...,3,0.0,True,mill's career way well bentham like joseph smi...,0.338,0.662,0.000,0.5574,69,11
2,"Mine uses a strait razor, and as much as i lov...",0,1420070400,1,vhisic,NaN,False,AdviceAnimals,NaN,NaN,...,1,0.0,True,mine use strait razor much love clipper love r...,0.363,0.563,0.074,0.8481,109,21
3,"Very fast, thank you!",0,1420070400,2,Mastersimpson,NaN,False,freedonuts,NaN,NaN,...,2,0.0,True,fast thank,0.714,0.286,0.000,0.3612,10,2
4,"The guy is a professional, and very good at wh...",0,1420070400,6,BigGupp1,NaN,False,WTF,NaN,NaN,...,6,0.0,True,guy professional good highly doubt miss often,0.251,0.346,0.404,-0.1953,45,7


## Feature Selection

In [3]:
# create doc2vec vector columns
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df["body_clean"].apply(lambda x: str(x).split(" ")))]

# train a Doc2Vec model with our text data
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

# transform each document into a vector data
doc2vec_df = df["body_clean"].apply(lambda x: model.infer_vector(str(x).split(" "))).apply(pd.Series)
doc2vec_df.columns = ["doc2vec_vector_" + str(x) for x in doc2vec_df.columns]
model_df = pd.concat([df, doc2vec_df], axis=1)


The next step consist in extracting vector representations for every comment. The module Gensim creates a numerical vector representation of every word in the corpus by using the contexts in which they appear (Word2Vec). This is performed using shallow neural networks. What's interesting is that similar words will have similar representation vectors.

Each text can also be transformed into numerical vectors using the word vectors (Doc2Vec). Same texts will also have similar representations and that is why we can use those vectors as training features.

We first have to train a Doc2Vec model by feeding in our text data. By applying this model on our comment, we can get those representation vectors.

In [4]:
model_df.head()

,body,downs,created_utc,score,author,distinguished,archived,subreddit,author_flair_css_class,author_flair_text,...,neutral_sentiment,negative_sentiment,compound_sentiment,nb_chars,nb_words,doc2vec_vector_0,doc2vec_vector_1,doc2vec_vector_2,doc2vec_vector_3,doc2vec_vector_4
0,Most of us have some family members like this....,0,1420070400,14,YoungModern,NaN,False,exmormon,NaN,NaN,...,0.375,0.000,0.6124,30,5,0.055995,0.076622,0.069184,0.064952,0.040338
1,But Mill's career was way better. Bentham is l...,0,1420070400,3,RedCoatsForever,NaN,False,CanadaPolitics,on,Ontario,...,0.662,0.000,0.5574,69,11,0.037877,0.218856,-0.094495,0.043705,0.207068
2,"Mine uses a strait razor, and as much as i lov...",0,1420070400,1,vhisic,NaN,False,AdviceAnimals,NaN,NaN,...,0.563,0.074,0.8481,109,21,0.084206,-0.016747,0.020130,0.109601,0.011560
3,"Very fast, thank you!",0,1420070400,2,Mastersimpson,NaN,False,freedonuts,NaN,NaN,...,0.286,0.000,0.3612,10,2,0.035912,0.041719,-0.071770,0.055905,0.005814
4,"The guy is a professional, and very good at wh...",0,1420070400,6,BigGupp1,NaN,False,WTF,NaN,NaN,...,0.346,0.404,-0.1953,45,7,0.105689,-0.009693,0.033573,-0.068896,-0.197156


In [5]:
model_df.shape

(178525, 26)

### Splitting, Cross Validation and Model Building

In [6]:
from sklearn.model_selection import train_test_split , cross_val_score , RandomizedSearchCV
from sklearn.preprocessing import StandardScaler ,MinMaxScaler ,LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [8]:
label = "score"
ignore_cols = [label, "body", "body_clean",'author', 'author_flair_css_class', 'author_flair_text', 'subreddit','distinguished']
features = [c for c in model_df.columns if c not in ignore_cols]
# to impute any missing values
it=IterativeImputer(random_state= 100)
model_df[features]=pd.DataFrame(it.fit_transform(model_df[features]))

X=model_df[features]
y=model_df[label]

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 100)

We first choose which features we want to use to train our model. Then we split our data into two parts:

1. one to train our model
2. one to assess its performances



## Model - RandomForest

In [24]:
result = cross_val_score(rf ,X,y, cv =10 ,scoring = 'accuracy',n_jobs =-1 , verbose =1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished


In [25]:
print(result)

[0.45331317 0.47879908 0.46586008 0.4538733  0.45342519 0.46062066
 0.45462693 0.48252297 0.48319516 0.45731571]


In [26]:
print(result.mean())

0.4643552247881312


In [10]:
rf = RandomForestClassifier()
rf.fit(X_train , y_train)

RandomForestClassifier()

In [11]:
rf_pred = rf.predict(X_test)

In [12]:
print("Acc of RandomForest without HyperParameter tuning is " , accuracy_score(rf_pred , y_test))

Acc of RandomForest without HyperParameter tuning is  0.9398123512113149


## Model - XGBoostClassifier

In [ ]:
xgb = XGBClassifier()
result1 = cross_val_score(xgb ,X,y, cv =10 ,scoring = 'accuracy',n_jobs =-1 , verbose =1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


In [ ]:
print(result1)

In [ ]:
print(result1.mean())

In [ ]:
print(xgb.fit(X_train , y_train))
xgb_pred = xgb.predict(X_test)

In [ ]:
print("Acc of XGBClassifier without HyperParameter tuning is " , accuracy_score(xgb_pred , y_test))

## HyperParam tuning for RandomForest

In [16]:
rf_param = {'n_estimators':list(range(100,500)) , 
         'max_depth':list(range(1,10)) , 
         'criterion':['gini','entropy'] ,
         'max_samples':list(range(1,10))    
}

In [17]:
rscv = RandomizedSearchCV(rf ,param_distributions=rf_param ,  cv =5 , n_iter=10 , scoring = 'accuracy',n_jobs =-1 , verbose =10)

In [18]:
rscv.fit(X,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done  33 out of  50 | elapsed: 16.6min remaining:  8.5min
[Parallel(n_jobs=-1)]: Done  39 out of  50 | elapsed: 19.4min remaining:  5.5min
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed: 20.8min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 21.7min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8,
                                                      9],
                                        'max_samples': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9],
                                        'n_estimators': [100, 101, 102, 103,
                                                         104, 105, 106, 107,
                                                         108, 109, 110, 111,
                                                         112, 113, 114, 115,
                                                         116, 117, 118, 119,
                                                         120, 121, 122, 123,
                                                         124, 125, 126, 127,
                                       

In [19]:
print(rscv.best_score_)
print()
print(rscv.best_estimator_)
print()
print(rscv.best_index_)
print()
print(rscv.best_params_)

0.46145357793026187
RandomForestClassifier(criterion='entropy', max_depth=5, max_samples=9,
                       n_estimators=228)
7
{'n_estimators': 228, 'max_samples': 9, 'max_depth': 5, 'criterion': 'entropy'}


In [22]:
# Fit and predict
rf = RandomForestClassifier(criterion='entropy', max_depth=5, max_samples=9,
                       n_estimators=228)
print(rf.fit(X_train , y_train))
print("Accuracy score of RF after tuning is :", accuracy_score(rf.predict(X_test) , y_test))

RandomForestClassifier(criterion='entropy', max_depth=5, max_samples=9,
                       n_estimators=228)
Accuracy score of RF after tuning is : 0.49105167343509315


## HyperParam Tuning for XGBoost classifier

In [ ]:
xgb_param = {'n_estimators':list(range(100,500)) , 
         'max_depth':list(range(1,10)) , 
         'learning_rate':[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.05,0.09] ,
         'min_child_weight ':list(range(1,10))    
}

In [ ]:
rscv = RandomizedSearchCV(xgb ,param_distributions=xgb_param ,  cv =5 , n_iter=10 , scoring = 'accuracy',n_jobs =-1 , verbose =10)
rscv.fit(X,y)

In [ ]:
print(rscv.best_score_)
print()
print(rscv.best_estimator_)
print()
print(rscv.best_index_)
print()
print(rscv.best_params_)

In [ ]:
# Fit and predict
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.09, max_delta_step=0, max_depth=2,
               min_child_weight =9,
              monotone_constraints='()', n_estimators=448, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

print(xgb.fit(X_train , y_train))
print("Accuracy score of XGB after tuning is :", accuracy_score(xgb.predict(X_test) , y_test))

# Conclusion

It is completely possible to use only raw text as input for making predictions. The most important thing is to be able to extract the relevant features from this raw source of data. This kind of data can often come as a good complementary source in data science projects in order to extract more learning features and increase the predictive power of the models.